<a href="https://colab.research.google.com/github/jatinkarthik-tripathy/BatteryTimeSeries/blob/master/battery_timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import pandas as pd
import numpy as np
import os

# Load Data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Battery TimeSeries/timeseries_df.csv")
print(len(df))
df.drop("Unnamed: 0", axis=1, inplace=True)
print(df.head(10))

3422401
   step0  step1  step2  step3  step4  step5  step6  step7  step8  step9  y_data
0    0.0    0.0  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000   0.015
1    0.0    0.0  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000   0.025
2    0.0    0.0  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.025   0.035
3    0.0    0.0  0.000  0.000  0.000  0.000  0.000  0.000  0.025  0.035   0.045
4    0.0    0.0  0.000  0.000  0.000  0.000  0.000  0.025  0.035  0.045   0.055
5    0.0    0.0  0.000  0.000  0.000  0.000  0.025  0.035  0.045  0.055   0.065
6    0.0    0.0  0.000  0.000  0.000  0.025  0.035  0.045  0.055  0.065   0.074
7    0.0    0.0  0.000  0.000  0.025  0.035  0.045  0.055  0.065  0.074   0.084
8    0.0    0.0  0.000  0.025  0.035  0.045  0.055  0.065  0.074  0.084   0.094
9    0.0    0.0  0.025  0.035  0.045  0.055  0.065  0.074  0.084  0.094   0.104


In [ ]:
from sklearn.model_selection import train_test_split

X_data = df.drop("y_data", axis=1)
y_data = df["y_data"]
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.33, random_state=42)
print(X_train.head())
print(X_train.shape)

         step0  step1  step2  step3  step4  step5  step6  step7  step8  step9
2608231  1.338  1.388  1.438  1.487  1.537  1.586  1.635  1.685  1.734  1.783
873323   1.412  1.464  1.516  1.567  1.619  1.670  1.721  1.773  1.824  1.875
3345811  3.856  3.902  3.948  3.994  4.040  4.086  4.132  4.178  4.223  4.269
1573090  6.504  6.550  6.597  6.642  6.688  6.734  6.780  6.826  6.871  6.917
988000   2.257  2.266  2.275  2.284  2.293  2.302  2.311  2.320  2.329  2.337
(2293008, 10)


# 4xLSTM 10 EPOCHS


In [ ]:
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as TFL
import tensorflow as tf
model = Sequential(name="4xLSTM")
model.add(TFL.Input(shape=(10, 1), name="Input"))
model.add(TFL.LSTM(128, name="LSTM0",  return_sequences=True))
model.add(TFL.LSTM(128, name="LSTM1", return_sequences=True))
model.add(TFL.LSTM(128, name="LSTM2",  return_sequences=True))
model.add(TFL.LSTM(128, name="LSTM3"))
model.add(TFL.Dense(1, activation='relu', name="Output"))
model.compile(loss="mse", 
              optimizer=tf.keras.optimizers.Adam(), 
              metrics=[tf.metrics.MeanAbsoluteError()])

print(model.summary())

Model: "4xLSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM0 (LSTM)                 (None, 10, 128)           66560     
_________________________________________________________________
LSTM1 (LSTM)                 (None, 10, 128)           131584    
_________________________________________________________________
LSTM2 (LSTM)                 (None, 10, 128)           131584    
_________________________________________________________________
LSTM3 (LSTM)                 (None, 128)               131584    
_________________________________________________________________
Output (Dense)               (None, 1)                 129       
Total params: 461,441
Trainable params: 461,441
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
checkpoint_path = "/content/drive/MyDrive/Battery TimeSeries/4xLSTM-10EPOCHS/checkpoints/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=0)

logdir = "/content/drive/MyDrive/Battery TimeSeries/4xLSTM-10EPOCHS/logs/"
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, 
                             verbose=1, mode='auto', cooldown=30, min_lr=0.0001)




model.fit(np.array(X_train).reshape(-1, X_train.shape[1], 1), y_train, batch_size=32, epochs=10, callbacks=[tensorboard_callback, cp_callback, reduce_lr])

In [ ]:
model.evaluate(np.array(X_test).reshape(-1, X_test.shape[1], 1), y_test, batch_size=32)

35294/35294 [==============================] - 144s 4ms/step - loss: 0.1063 - mean_absolute_error: 0.0461


[0.10634442418813705, 0.04608534276485443]

In [ ]:
model_name = "4xLSTM-10EPOCHS.h5"
model.save(f"/content/drive/MyDrive/Battery TimeSeries/4xLSTM-10EPOCHS/{model_name}")

#8xLSTM [256 units]

In [ ]:
model = Sequential(name="4xLSTM")
model.add(TFL.Input(shape=(10, 1), name="Input"))
model.add(TFL.LSTM(256, name="LSTM0",  return_sequences=True))
model.add(TFL.LSTM(256, name="LSTM1", return_sequences=True))
model.add(TFL.LSTM(256, name="LSTM2",  return_sequences=True))
model.add(TFL.LSTM(256, name="LSTM3",  return_sequences=True))
model.add(TFL.LSTM(256, name="LSTM4",  return_sequences=True))
model.add(TFL.LSTM(256, name="LSTM5",  return_sequences=True))
model.add(TFL.LSTM(256, name="LSTM6",  return_sequences=True))
model.add(TFL.LSTM(256, name="LSTM7"))
model.add(TFL.Dense(1, activation='relu', name="Output"))
model.compile(loss="mse", 
              optimizer=tf.keras.optimizers.Adam(), 
              metrics=[tf.metrics.MeanAbsoluteError()])

print(model.summary())

Model: "4xLSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM0 (LSTM)                 (None, 10, 256)           264192    
_________________________________________________________________
LSTM1 (LSTM)                 (None, 10, 256)           525312    
_________________________________________________________________
LSTM2 (LSTM)                 (None, 10, 256)           525312    
_________________________________________________________________
LSTM3 (LSTM)                 (None, 10, 256)           525312    
_________________________________________________________________
LSTM4 (LSTM)                 (None, 10, 256)           525312    
_________________________________________________________________
LSTM5 (LSTM)                 (None, 10, 256)           525312    
_________________________________________________________________
LSTM6 (LSTM)                 (None, 10, 256)           52531

In [ ]:
checkpoint_path = "/content/drive/MyDrive/Battery TimeSeries/8xLSTM-25EPOCHS/checkpoints/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=0)

logdir = "/content/drive/MyDrive/Battery TimeSeries/8xLSTM-25EPOCHS/logs/"
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, 
                             verbose=1, mode='auto', cooldown=30, min_lr=0.0001)

model.fit(np.array(X_train).reshape(-1, X_train.shape[1], 1), y_train, batch_size=64, epochs=25, callbacks=[tensorboard_callback, cp_callback, reduce_lr])

Epoch 1/25
35829/35829 [==============================] - 582s 15ms/step - loss: 0.6908 - mean_absolute_error: 0.2811
Epoch 2/25
35829/35829 [==============================] - 488s 14ms/step - loss: 0.1757 - mean_absolute_error: 0.1137
Epoch 3/25
35829/35829 [==============================] - 489s 14ms/step - loss: 0.1355 - mean_absolute_error: 0.0850
Epoch 4/25
35829/35829 [==============================] - 490s 14ms/step - loss: 0.1210 - mean_absolute_error: 0.0712
Epoch 5/25
35829/35829 [==============================] - 489s 14ms/step - loss: 0.1353 - mean_absolute_error: 0.0687
Epoch 6/25
35829/35829 [==============================] - 488s 14ms/step - loss: 0.1346 - mean_absolute_error: 0.0688
Epoch 7/25
35829/35829 [==============================] - 485s 14ms/step - loss: 0.1158 - mean_absolute_error: 0.0645
Epoch 8/25
35829/35829 [==============================] - 486s 14ms/step - loss: 0.1126 - mean_absolute_error: 0.0605
Epoch 9/25
35829/35829 [==============================] 

In [ ]:
model.evaluate(np.array(X_test).reshape(-1, X_test.shape[1], 1), y_test, batch_size=32)

35294/35294 [==============================] - 235s 7ms/step - loss: 0.1090 - mean_absolute_error: 0.0533


[0.109040766954422, 0.05333847552537918]

In [ ]:
model_name = "8xLSTM-25EPOCHS.h5"
model.save(f"/content/drive/MyDrive/Battery TimeSeries/8xLSTM-25EPOCHS/{model_name}")